In [28]:
import copy
import glob
import importlib
import itertools
import math
import operator
import os
import random
import time
from os.path import *
import psutil
import shutil

import keras.layers as layers
import keras.models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import cnn_builder as cbuild
import cnn_runner as crun
import config
import ddpg.env as denv
import ddpg.main as qmain
import ddpg.learning_nets as ln
import ddpg.task_nets as tn
import dr_methods as drm
import seg_methods as sm
import niftiutils.helper_fxns as hf
import niftiutils.masks as masks
import niftiutils.transforms as tr
import niftiutils.visualization as vis
import voi_methods as vm

%matplotlib inline
np.set_printoptions(3, suppress=True)
#np.random.seed(0)

In [2]:
def memory():
    pid = os.getpid()
    py = psutil.Process(pid)
    gigs = py.memory_info()[0]/2.**30
    print('Memory use:', gigs)
    return gigs

In [3]:
importlib.reload(cbuild)
C = config.Config()
dqn_generator = cbuild._train_gen_ddpg()

In [ ]:
# importlib.reload(qmain)
Q = qmain.CRSNet(1.)
Q.train(dqn_generator, verbose=True)

In [6]:
Q.save_models()

In [ ]:
shutil.rmtree(C.model_dir)

In [7]:
os.makedirs(C.model_dir)

In [24]:
cropI = np.random.normal(size=(1,*C.dims,3))

crop_true_seg = np.random.normal(size=(1,*C.dims,3))
true_cls = np.random.uniform(size=(1,3))

y_true = np.random.normal(size=(1,*C.dims,3))
y_pred = np.random.normal(size=(1,*C.dims,4))

loss_layer = train_model.layers[-1]

In [25]:
for _ in range(10):
    print(loss_layer.get_loss(0, y_true, y_pred, C.loss_weights[1:]))
    memory();

-0.05015051
Memory use: 2.0087203979492188
-0.050445613
Memory use: 2.0087203979492188
-0.050510425
Memory use: 2.008716583251953
-0.049581293
Memory use: 2.008716583251953
-0.049884304
Memory use: 2.008716583251953
-0.048125546
Memory use: 2.008716583251953
-0.049017023
Memory use: 2.008716583251953
-0.04843785
Memory use: 2.0087203979492188
-0.050015423
Memory use: 2.0087203979492188
-0.049520925
Memory use: 2.0087203979492188


In [ ]:
importlib.reload(tn)
for _ in range(10):
    log_vars = [K.get_value(x)[0] for x in train_model.layers[-1].log_vars]
    loss = np.sum(np.exp(-log_vars[0]) * K.get_value(tn.hetero_cls_loss(y_true, y_pred)) + log_vars[0], -1)
    print(np.mean(loss))
    memory();

In [30]:
A = prediction_model.predict(cropI)

In [ ]:
for _ in range(10):
    train_model.train_on_batch([cropI, crop_true_seg, true_cls], None)
    A=prediction_model.predict(cropI);
    print(A[1])
    memory();

In [ ]:
importlib.reload(qmain)
Q = qmain.CRSNet(1.)

replay_conf = {'size': 10000,
        'learn_start': 100,
        'partition_num': 100,
        'total_step': 10000,
        'batch_size': 4}
BATCH_SIZE = replay_conf["batch_size"]

Q.load_models(replay_conf)

In [ ]:
Q.critic.model.summary()

In [ ]:
Q.actor.model.summary()

In [ ]:
Q.env.pred_model.summary()

In [ ]:
importlib.reload(qmain)
Q = qmain.CRSNet(.5)
img, true_seg, true_cls = next(dqn_generator)
seg, seg_var, cls = Q.run(img)

In [ ]:
vis.draw_slices(img)

In [ ]:
vis.draw_slices(true_seg)

In [ ]:
vis.draw_slices(seg, normalize=[0,.05])

In [ ]:
hist = M.fit_generator(gen, steps_per_epoch=2, epochs=2)

prediction_model = get_prediction_model()
trainable_model = get_trainable_model(prediction_model)
trainable_model.compile(optimizer='adam', loss=None)
assert len(trainable_model.layers[-1].trainable_weights) == 2  # two log_vars, one for each output
assert len(trainable_model.losses) == 1
hist = trainable_model.fit([X, Y1, Y2], nb_epoch=nb_epoch, batch_size=batch_size, verbose=0)

[np.exp(K.get_value(log_var[0]))**0.5 for log_var in trainable_model.layers[-1].log_vars]

In [ ]:
importlib.reload(config)
importlib.reload(hf)
importlib.reload(cbuild)
importlib.reload(crun)
C = config.Config('etiology')
T = config.Hyperparams()
T.get_best_hyperparams()
T.epochs = 30
T.steps_per_epoch = 300

In [ ]:
importlib.reload(cbuild)
importlib.reload(dqna)
#dqn_generator = cbuild._train_gen_dqn([])
agent = dqna.train_dqn(dqn_generator, agent)

In [ ]:
agent.cls_model.fit(cls_generator)

In [ ]:
T.padding = ['same', 'same']
T.pool_sizes = [(2,2,2),(2,2,2)]
T.f = [64,64,64,64,64,64,64]
T.skip_con = True
T.epochs = 20

In [ ]:
drm.dcm2npy_batch(acc_nums=["E100113043"])
#vm.reset_accnum('E105464882')

In [ ]:
vm.plot_check(2, "E102088195");

In [ ]:
vm.xref_dirs_with_excel()

In [ ]:
crun.run_fixed_hyperparams([C], hyperparams=T)#C_list)

In [ ]:
importlib.reload(cbuild)
model = cbuild.build_cnn_hyperparams(T)
model.summary(line_length=120)

In [ ]:
importlib.reload(cbuild)
#Z_reader = ['E103312835_1','12823036_0','12569915_0','E102093118_0','E102782525_0','12799652_0','E100894274_0','12874178_3','E100314676_0','12842070_0','13092836_2','12239783_0','12783467_0','13092966_0','E100962970_0','E100183257_1','E102634440_0','E106182827_0','12582632_0','E100121654_0','E100407633_0','E105310461_0','12788616_0','E101225606_0','12678910_1','E101083458_1','12324408_0','13031955_0','E101415263_0','E103192914_0','12888679_2','E106096969_0','E100192709_1','13112385_1','E100718398_0','12207268_0','E105244287_0','E102095465_0','E102613189_0','12961059_0','11907521_0','E105311123_0','12552705_0','E100610622_0','12975280_0','E105918926_0','E103020139_1','E101069048_1','E105427046_0','13028374_0','E100262351_0','12302576_0','12451831_0','E102929168_0','E100383453_0','E105344747_0','12569826_0','E100168661_0','12530153_0','E104697262_0']
X_test, Y_test, train_generator, num_samples, train_orig, Z = cbuild.get_cnn_data(n=4)#, Z_test_fixed=Z_reader)
Z_test, Z_train_orig = Z
X_train_orig, Y_train_orig = train_orig
hist = model.fit_generator(train_generator, steps_per_epoch=T.steps_per_epoch, epochs=T.epochs, validation_data=[X_test, Y_test])#, callbacks=[T.early_stopping])

In [ ]:
model.save(join(C.model_dir, "model_.hdf5"))

In [ ]:
model.predict(X_train_orig[20:40])

In [ ]:
Y_train_orig[:20]

In [ ]:
hist = model.fit_generator(train_generator, steps_per_epoch=T.steps_per_epoch, epochs=T.epochs, validation_data=[X_test, Y_test])#, callbacks=[T.early_stopping])

In [ ]:
#plot_with_bbox(fn_list[2], cls_mapping[wrong_guesses[2]])
Y_pred = model.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

#save_output(Z_test, y_pred, y_true)

print(accuracy_score(y_true, y_pred))
#y_true_simp, y_pred_simp, _ = cnna.merge_classes(y_true, y_pred)
#print(accuracy_score(y_true_simp, y_pred_simp))